In [2]:
%pip install flask-cors

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from joblib import dump, load
from flask import Flask, request, jsonify
from flask_cors import CORS
import logging

app = Flask(__name__)
CORS(app)

logging.basicConfig(level=logging.INFO)

pipeline = None
le_med = LabelEncoder()
le_diet = LabelEncoder()

def train_and_save_model():
    """Train and save the ML pipeline and label encoders"""
    global pipeline, le_med, le_diet
    
    try:

        df = pd.read_csv("clean_symptom_data.csv", skiprows=1)
        df.columns = ['Symptoms', 'Recommended Medicine', 'Diet Recommendation']
        df = df.applymap(lambda x: str(x).strip(' "'))
        df.dropna(inplace=True)
        
        X = df['Symptoms']
        y_medicine = le_med.fit_transform(df['Recommended Medicine'])
        y_diet = le_diet.fit_transform(df['Diet Recommendation'])
        Y = np.vstack((y_medicine, y_diet)).T

        X_train, X_test, Y_train, Y_test = train_test_split(
            X, Y, test_size=0.2, random_state=42
        )

        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english')),
            ('clf', MultiOutputClassifier(
                RandomForestClassifier(n_estimators=100, random_state=42)
            ))
        ])
        
        pipeline.fit(X_train, Y_train)
        app.logger.info("Model training completed")

        dump(pipeline, 'medicine_pipeline.joblib')
        dump(le_med, 'label_encoder_med.joblib')
        dump(le_diet, 'label_encoder_diet.joblib')
        app.logger.info("Model artifacts saved")

    except Exception as e:
        app.logger.error(f"Model training failed: {str(e)}")
        raise

def load_model():
    """Load trained model artifacts"""
    global pipeline, le_med, le_diet
    try:
        pipeline = load('medicine_pipeline.joblib')
        le_med = load('label_encoder_med.joblib')
        le_diet = load('label_encoder_diet.joblib')
        app.logger.info("Model loaded successfully")
    except Exception as e:
        app.logger.error(f"Model loading failed: {str(e)}")
        raise

def initialize_model():
    """Initialize or train the model"""
    try:
        load_model()
    except Exception as e:
        app.logger.warning("Training new model...")
        train_and_save_model()
        load_model()

with app.app_context():
    initialize_model()

@app.route('/predict', methods=['POST'])
def predict():
    """Handle prediction requests"""
    try:
        data = request.get_json()
        
        if not data or 'symptoms' not in data:
            return jsonify({'error': 'Invalid request format'}), 400
            
        symptoms_list = data['symptoms']
        
        if not symptoms_list or not isinstance(symptoms_list, list):
            return jsonify({'error': 'Symptoms must be a non-empty list'}), 400
            
        symptoms_text = ", ".join(symptoms_list).lower()
        
        if not pipeline or not hasattr(pipeline, 'predict'):
            return jsonify({'error': 'Model not initialized'}), 500
   
        pred = pipeline.predict([symptoms_text])[0]
        medicine = le_med.inverse_transform([pred[0]])[0]
        diet = le_diet.inverse_transform([pred[1]])[0]
        
        return jsonify({
            'medicine': medicine,
            'diet': diet
        })
        
    except Exception as e:
        app.logger.error(f"Prediction error: {str(e)}")
        return jsonify({
            'error': 'Prediction failed',
            'message': str(e)
        }), 500

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'model_loaded': pipeline is not None
    })

if __name__ == '__main__':
    with app.app_context():
        initialize_model()
        
    app.run(host='0.0.0.0', port=5000, debug=False)

INFO:__main__:✅ Model loaded successfully
INFO:__main__:✅ Model loaded successfully


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.43.5:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/May/2025 06:35:39] "POST /predict HTTP/1.1" 200 -


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier  
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from joblib import dump, load
from flask import Flask, request, jsonify
from flask_cors import CORS
import logging

app = Flask(__name__)
CORS(app)

logging.basicConfig(level=logging.INFO)

pipeline = None
le_med = LabelEncoder()
le_diet = LabelEncoder()

def train_and_save_model():
    """Train and save the ML pipeline and label encoders"""
    global pipeline, le_med, le_diet
    
    try:
        df = pd.read_csv("clean_symptom_data.csv", skiprows=1)
        df.columns = ['Symptoms', 'Recommended Medicine', 'Diet Recommendation']
        df = df.applymap(lambda x: str(x).strip(' "'))
        df.dropna(inplace=True)
        
        X = df['Symptoms']
        y_medicine = le_med.fit_transform(df['Recommended Medicine'])
        y_diet = le_diet.fit_transform(df['Diet Recommendation'])
        Y = np.vstack((y_medicine, y_diet)).T

        X_train, X_test, Y_train, Y_test = train_test_split(
            X, Y, test_size=0.2, random_state=42
        )

        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english')),
            ('clf', MultiOutputClassifier(
                GradientBoostingClassifier(n_estimators=100, random_state=42)  # CHANGED MODEL
            ))
        ])
        
        pipeline.fit(X_train, Y_train)
        app.logger.info("Model training completed")

        dump(pipeline, 'medicine_pipeline.joblib')
        dump(le_med, 'label_encoder_med.joblib')
        dump(le_diet, 'label_encoder_diet.joblib')
        app.logger.info("Model artifacts saved")

    except Exception as e:
        app.logger.error(f"Model training failed: {str(e)}")
        raise

def load_model():
    """Load trained model artifacts"""
    global pipeline, le_med, le_diet
    try:
        pipeline = load('medicine_pipeline.joblib')
        le_med = load('label_encoder_med.joblib')
        le_diet = load('label_encoder_diet.joblib')
        app.logger.info("Model loaded successfully")
    except Exception as e:
        app.logger.error(f"Model loading failed: {str(e)}")
        raise

def initialize_model():
    """Initialize or train the model"""
    try:
        load_model()
    except Exception as e:
        app.logger.warning("Training new model...")
        train_and_save_model()
        load_model()

with app.app_context():
    initialize_model()

@app.route('/predict', methods=['POST'])
def predict():
    """Handle prediction requests"""
    try:
        data = request.get_json()
        
        if not data or 'symptoms' not in data:
            return jsonify({'error': 'Invalid request format'}), 400
            
        symptoms_list = data['symptoms']
        
        if not symptoms_list or not isinstance(symptoms_list, list):
            return jsonify({'error': 'Symptoms must be a non-empty list'}), 400
            
        symptoms_text = ", ".join(symptoms_list).lower()
        
        if not pipeline or not hasattr(pipeline, 'predict'):
            return jsonify({'error': 'Model not initialized'}), 500
   
        pred = pipeline.predict([symptoms_text])[0]
        medicine = le_med.inverse_transform([pred[0]])[0]
        diet = le_diet.inverse_transform([pred[1]])[0]
        
        return jsonify({
            'medicine': medicine,
            'diet': diet
        })
        
    except Exception as e:
        app.logger.error(f"Prediction error: {str(e)}")
        return jsonify({
            'error': 'Prediction failed',
            'message': str(e)
        }), 500

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'model_loaded': pipeline is not None
    })

if __name__ == '__main__':
    with app.app_context():
        initialize_model()
        
    app.run(host='0.0.0.0', port=5000, debug=False)

INFO:__main__:✅ Model loaded successfully
INFO:__main__:✅ Model loaded successfully


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.43.5:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/May/2025 06:39:01] "POST /predict HTTP/1.1" 200 -


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from joblib import dump, load
from flask import Flask, request, jsonify
from flask_cors import CORS
import logging

app = Flask(__name__)
CORS(app)

logging.basicConfig(level=logging.INFO)

pipeline = None
le_med = LabelEncoder()
le_diet = LabelEncoder()

def train_and_save_model():
    """Train and save the ML pipeline and label encoders"""
    global pipeline, le_med, le_diet
    
    try:
        df = pd.read_csv("clean_symptom_data.csv", skiprows=1)
        df.columns = ['Symptoms', 'Recommended Medicine', 'Diet Recommendation']
        df = df.applymap(lambda x: str(x).strip(' "'))
        df.dropna(inplace=True)
        
        X = df['Symptoms']
        y_medicine = le_med.fit_transform(df['Recommended Medicine'])
        y_diet = le_diet.fit_transform(df['Diet Recommendation'])
        Y = np.vstack((y_medicine, y_diet)).T

        X_train, X_test, Y_train, Y_test = train_test_split(
            X, Y, test_size=0.2, random_state=42
        )

        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english')),
            ('clf', MultiOutputClassifier(
                XGBClassifier(
                    n_estimators=300,
                    learning_rate=0.1,
                    max_depth=5,
                    subsample=0.8,
                    eval_metric='mlogloss',
                    use_label_encoder=False,
                    random_state=42
                )
            ))
        ])
        
        pipeline.fit(X_train, Y_train)
        app.logger.info("✅ Model training completed")

        dump(pipeline, 'medicine_pipeline.joblib')
        dump(le_med, 'label_encoder_med.joblib')
        dump(le_diet, 'label_encoder_diet.joblib')
        app.logger.info("💾 Model artifacts saved")

    except Exception as e:
        app.logger.error(f"Model training failed: {str(e)}")
        raise

def load_model():
    """Load trained model artifacts"""
    global pipeline, le_med, le_diet
    try:
        pipeline = load('medicine_pipeline.joblib')
        le_med = load('label_encoder_med.joblib')
        le_diet = load('label_encoder_diet.joblib')
        app.logger.info("Model loaded successfully")
    except Exception as e:
        app.logger.error(f"Model loading failed: {str(e)}")
        raise

def initialize_model():
    """Initialize or train the model"""
    try:
        load_model()
    except Exception as e:
        app.logger.warning("Training new model...")
        train_and_save_model()
        load_model()

with app.app_context():
    initialize_model()

@app.route('/predict', methods=['POST'])
def predict():
    """Handle prediction requests"""
    try:
        data = request.get_json()
        
        if not data or 'symptoms' not in data:
            return jsonify({'error': 'Invalid request format'}), 400
            
        symptoms_list = data['symptoms']
        
        if not symptoms_list or not isinstance(symptoms_list, list):
            return jsonify({'error': 'Symptoms must be a non-empty list'}), 400
            
        symptoms_text = ", ".join(symptoms_list).lower()
        
        if not pipeline or not hasattr(pipeline, 'predict'):
            return jsonify({'error': 'Model not initialized'}), 500
   
        pred = pipeline.predict([symptoms_text])[0]
        medicine = le_med.inverse_transform([pred[0]])[0]
        diet = le_diet.inverse_transform([pred[1]])[0]
        
        return jsonify({
            'medicine': medicine,
            'diet': diet
        })
        
    except Exception as e:
        app.logger.error(f"Prediction error: {str(e)}")
        return jsonify({
            'error': 'Prediction failed',
            'message': str(e)
        }), 500

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'model_loaded': pipeline is not None
    })

if __name__ == '__main__':
    with app.app_context():
        initialize_model()
        
    app.run(host='0.0.0.0', port=5000, debug=False)

INFO:__main__:Model loaded successfully
INFO:__main__:Model loaded successfully


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.43.5:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/May/2025 07:05:35] "POST /predict HTTP/1.1" 200 -
